In [40]:
import pandas as pd
import pymongo
from functools import reduce

In [41]:
#get data from website
ship_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_shipwrecks_of_the_United_States')

#combine tables
ship_wreck_data = pd.concat(ship_df)

#drop Nans
ship_wreck_data = ship_wreck_data.dropna()

#drop duplicates
ship_wreck_data = ship_wreck_data.drop_duplicates()

#name columns
ship_wreck_data.columns = ['ship','sunk_date','notes','coordinates']

#drop the first row of data because it does not needed
ship_wreck_data = ship_wreck_data.drop([0])

#reset index
ship_wreck_data = ship_wreck_data.reset_index(drop=True)

In [43]:
true_coordinates = []
#ship_wreck_data1 = ship_wreck_data.drop([0,74])

for coordinate in ship_wreck_data['coordinates']:
    if coordinate.find('/')==-1:
        print(coordinate)
    else:
        latlong = coordinate.split('/')[1]
        new_latlong = latlong.replace("°","").strip()
        lat = new_latlong.split()[0]
        long = new_latlong.split()[1]
        lat = lat[1:]
    
        #print(lat[:len(lat)-2])
        if lat[len(lat)-1] == 'S':
            lat = 0-float(lat[:len(lat)-2])
           # print(lat)
        else:
            lat = lat.strip("N")
            #print(lat)
        if long[len(long)-1] == 'W':
            long = 0-float(long[:len(long)-2])
            #print(long)
        else:
            long = long.strip("E")
        #print(long)
        try:
            clean_coordinates = [float(lat),float(long)]
            print(clean_coordinates)
            true_coordinates.append(clean_coordinates)
        except:
            print("Sorry")

[32.24359, -88.0123]
[30.76924, -88.0205]
[30.3835, -88.033]
[30.60231, -88.0386]
[30.23167, -88.0258]
[30.76924, -88.0205]
[53.883, -169.96]
[57.48, 153.84]
[52.23778, 177.41806]
[53.267, 174.4]
[52.542, 172.177]
[58.3765, -134.788]
[52.25, 173.85]
[52.283, 178.133]
[58.234, -134.46]
[58.6022, -135.023]
[55.883, -136.45]
[41.17367, -73.1856]
[41.17367, -73.1856]
[38.84, -74.0]
[32.08472, -81.0358]
[13.45917, 144.65417]
[21.364775, -157.95011]
[20.862722, -156.67519]
[21.36889, -157.9625]
[39.1736028, -94.670247]
[30.48722, -91.2013]
[29.580617, -92.65498]
[29.36333, -89.4613]
[28.617, -90.75]
[43.62283, -70.2079]
[43.5577, -70.162]
[38.84, -74.0]
[38.13611, -76.5527]
[46.79, -85.6]
[47.14417, -88.4597]
[46.71278, -92.0286]
[47.27139, -91.2716]
[47.20611, -91.358]
[46.80333, -92.0111]
[46.84611, -91.7772]
[46.84722, -91.7288]
[46.78333, -92.0694]
[32.37583, -90.8666]
[38.611483, -90.1881]
[37.315, -89.5]
[38.79472, -90.4669]
[39.1332, -94.073]
[41.52333, -96.0288]
[40.1269, -73.656]
[4

In [44]:
true_coordinates_df = pd.DataFrame(true_coordinates)
true_coordinates_df.columns = ['Latitude', 'Longitude']

#drop duplicate values
true_coordinates_df =true_coordinates_df.drop_duplicates()

#convert true_coordinates_df to a list then into a series so both values will be in one column as coordinates
coordinate = true_coordinates_df.values.tolist()
coordinate_series = pd.Series(coordinate)

#append coordinate_series to coordinate column and drop Nan values
ship_wreck_df = ship_wreck_data
ship_wreck_df['coordinates'] = coordinate_series

In [47]:
#convert dataframe to json
import json
ship_wreck_json = ship_wreck_df.to_json(orient='records')
json_format = json.loads(ship_wreck_json)

In [48]:
import pymongo
#create mongo database
myclient = pymongo.MongoClient('mongodb://localhost:27017/')

db = myclient['shipwreckData']

In [49]:
json_data = db.shipwreckData
json_data.drop()
json_data.insert_many(json_format)